# Schnorr Signatures

we start by defining schnorr signatures on a cyclic group of order 23. This group might have come from an elliptic curve. In any case these signatures are not strong!
However we use such a small example to be able to actually comprehend all calculations. We would even be able to do them on a sheet of paper!

Learn more about this code in my educational video about Schnorr Signatures: https://youtu.be/n5aompcR9W0

In [1]:
import random

In [2]:
p=23
points = {i: chr(97+i) for i in range(p)}

In [3]:
print("private keys are numbers with 0 <= n < 23")
x_a = 17  # random.randint(0,22)
x_b = 15  # random.randint(0,22)

A = points[x_a]
B = points[x_b]

print("1st keypair: (x_a,A)=({},{})".format(x_a, A))
print("2nd keypair: (x_b,B)=({},{})".format(x_b, B))

private keys are numbers with 0 <= n < 23
1st keypair: (x_a,A)=(17,r)
2nd keypair: (x_b,B)=(15,p)


In [4]:
def H(string):
    """ poor hashfunction for Z/23Z

    I did not even check if it is a proper hash function. In general
    mod prime is not too bad. but since the alphabet has 26 letters
    we might run into some stupid collisions.
    Also since the space is so clear it is quite clear how results
    will change once we flip a letter.
    """
    res = 0
    for c in string:
        res += ord(c)
    return res % p

In [5]:
print("test our 'hash' function on two hashes...")
message = "Bitcoin Transaction"
print("message:", "\"{}\"".format(message), "hash:", H(message))
message = "Bitcoin transaction"
print("message:", "\"{}\"".format(message), "hash:", H(message))


message = "this is our message which should be signed"
print("message:", "\"{}\"".format(message), "hash:", H(message))

test our 'hash' function on two hashes...
message: "Bitcoin Transaction" hash: 16
message: "Bitcoin transaction" hash: 2
message: "this is our message which should be signed" hash: 7


In [6]:
 def sign(m, priv):
    r = random.randint(0, 22)
    print("  random nonce for signature:",r)
    return (r + H(m)*priv) % p, points[r]

print("compute partial signatures...")
s_a, R_a = sign(message, x_a)
s_b, R_b = sign(message, x_b)

compute partial signatures...
  random nonce for signature: 11
  random nonce for signature: 3


Our group is actualy a 1 dimentional vector space over the prime field F_{23}

So let us call elements in F_{23} scalars and define a scala multiplication

In [7]:
def add_points(a, b):
    """
    This is only working because we constructed our group in a way
    that we know the discret logarithm. This entire construction 
    MUST NOT be used and is only for educational purpose anyway.
    """
    a = ord(a)-97
    b = ord(b)-97
    res = a+b
    res = res % p
    return points[res]

def scalar_mult_point(scalar, point):
    """ this could also be way more efficient """
    if scalar == 0:
        scalar = p
    res = point
    for i in range(scalar-1):
        res = add_points(res, point)
    return res



def verify(s, R, m, pup):
    lhs = points[s]
    rhs = add_points(R, scalar_mult_point(H(m), pup))
    return lhs == rhs

In [8]:
print("Verify partial signatures")
print("(s_a,R_a) = ({},{}) is a {} signature for key A".format(s_a, R_a, verify(s_a, R_a, message, A)))
print("(s_b,R_b) = ({},{}) is a {} signature for key B".format(s_b, R_b, verify(s_b, R_b, message, B)))
print("(s_a,R_a) = ({},{}) is a {} signature for key A".format((s_a+1)%p, R_a, verify((s_a+1)%p, R_a, message, A)))
print("(s_a,R_a) = ({},{}) is a {} signature for key A".format(s_a, add_points(R_a,'c'), verify(s_a, add_points(R_a,'c'), message, A)))

Verify partial signatures
(s_a,R_a) = (15,l) is a True signature for key A
(s_b,R_b) = (16,d) is a True signature for key B
(s_a,R_a) = (16,l) is a False signature for key A
(s_a,R_a) = (15,n) is a False signature for key A
